In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson
from selenium.webdriver.chrome.options import Options
from multiprocessing.dummy import Pool

In [4]:
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    return response

In [5]:
def getHTML(url):
    response = getResponse(url)
    response.encoding = 'utf-8'
    return response.text

In [6]:
# 通过selenium获取page_text
def getHTMLBySelenium(url):
    chrome_options =Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(executable_path='C:/Users/Administrator/Desktop/chromedriver_win32/chromedriver.exe', options=chrome_options)
    browser.get(url)
    time.sleep(2)
    page_text = browser.page_source
    browser.quit()
    return page_text

In [ ]:
# 正则
# 使用聚焦爬虫将页面中所有的图片解析提取
# ex = '<div class="thumb">.*?<img src="(.*?)" alt.*?</div>'
# re.S单行匹配
# re.M多行匹配
# img_src_list = re.findall(ex, page_text, re.S)

In [ ]:
# 多线程
# pool = Pool(4)
# pool.map(video2audio, allVideo)
# pool.close()
# pool.join()

In [ ]:
# 多线程下载

In [1]:
def download(url):
    fileName = (url.rsplit('/', 1)[-1]).split('?', 1)[0]
    path = 'audio/'
    try:
        content = getResponse(url).content
        with open(path + fileName, 'wb') as fp:
            fp.write(content)
        print(fileName, 'successful')
    except Exception as e:
        print("error")

In [ ]:
allObj = []
for i, url in enumerate(srcUrls):
    obj = {
        'id':i,
        'url':url
    }
    allObj.append(obj)
print(len(allObj))

In [ ]:
def multithreadDownload(obj):
    ID = obj['id']
    url = obj['url']
    print(f'start download {ID}')
    download(url)

In [ ]:
# 爬取ts
def downloadEveryTs(url):
    allContent = getResponse(url).text
    fileLine = allContent.split('\n')
    downloadPath = 'download'
    if not os.path.exists(downloadPath):
        os.mkdir(downloadPath)
    # 通过判断文件头来确定是否是M3U8文件
    if fileLine[0] != "#EXTM3U":
        raise BaseException(u"非M3U8的链接")
    else:
        unknow = True# 用来判断是否找到了下载地址
        for index, line in enumerate(fileLine):
            if "EXTINF" in line:
                unknow = False
                # 拼出ts片段的URL
                tsUrl = url.rsplit('/', 1)[0] + '/' + fileLine[index + 1]# i.ts在下一行
                #print(tsUrl)
                res = getResponse(tsUrl)
                cFuleName = (str(fileLine[index + 1])).split('?')[0]
                with open(downloadPath + '/' + cFuleName, 'ab') as fp:
                    fp.write(res.content)
                    fp.flush()
        if unknow:
            raise BaseException(u"未找到对应的下载地址")
        else:
            print("ts 下载完成")

In [ ]:
# ts文件合并
def file_walker(path):
    file_list = []
    for root, dirs, files in os.walk(path): # 生成器
        for fn in files:
            p = str(root+'/'+fn)
            file_list.append(p)
 
    #print(file_list)
    return file_list
 
# def combine(ts_path, combine_path, file_name):
#     file_list = file_walker(ts_path)
#     file_path = combine_path + file_name + '.ts'
#     with open(file_path, 'wb+') as fw:
#         for i in range(len(file_list)): 
#             fw.write(open(file_list[i], 'rb').read())

def combine(ts_path, combine_path, file_name):
    file_list = file_walker(ts_path)
    cnt = 1
    Name = f'_{cnt}'
    file_path = combine_path + file_name + Name + '.ts'
    fp = open(file_path, 'wb+')
    for i in range(len(file_list)):
        fp.write(open(file_list[i], 'rb').read())
        if (i + 1) % 500 == 0:
            fp.close()
            cnt += 1
            Name = f'_{cnt}'
            file_path = combine_path + file_name + Name + '.ts'
            fp = open(file_path, 'wb+')
    fp.close()
    
            
# 删除之前下载的一段段的ts文件
def clear():
    shutil.rmtree('./download3')
    time.sleep(1)
    os.mkdir('download3')
    
def downloadTs(url, fileName):
    downloadEveryTs(url)
    combine('./download', 'E:/work/ts/', fileName)
    print(fileName, ' successful')
    clear()

In [ ]:
# 获取视频下载地址以及对应文本内容
def getMp4UrlAndText(url):
    page_text = getHTMLBySelenium(url)
    tree = etree.HTML(page_text)
    # https://d03.lawtimeimg.com/video/48/27676.mp4
    temp = tree.xpath('//div[@class="video-box"]/video/@src')
    if len(temp) == 0:
        return '', ''
    mp4Url = 'https:' + temp[0]
    allText = tree.xpath('//div[@class="video-text"]//text()')
    text = '\n'.join(allText)
    return mp4Url, text

In [ ]:
# 下载视频以及对应文本
def downloadVideoAndText(url, fileName):
    try:
        mp4Url, text = getMp4UrlAndText(url)
        if mp4Url == '':
            print(fileName, ' empty')
            return
        with open('text/' + fileName + '.txt', 'w', encoding='utf-8') as fp:
            fp.write(text)
            print(fileName, ' text successful')
        content = getResponse(mp4Url).content
        with open('mp4/' + fileName + '.mp4', 'wb') as fp:
            fp.write(content)
            print(fileName, ' mp4 successful')
    except Exception as e:
        print(fileName, 'error')

In [ ]:
# 针对验证码不需要对错 子需要Enter即可的情况
def retry(url):
    try:
        print('retry')
        browser = webdriver.Chrome(executable_path='C:/Users/Administrator/Desktop/chromedriver_win32/chromedriver.exe')
        browser.get(url)
        time.sleep(2)
        """
        inputLine = browser.find_element_by_id('vgcode')
        inputLine.send_keys('1234')
        btn = browser.find_element_by_id('resubimt')
        time.sleep(20)
        btn.click()
        time.sleep(60)
        """
        ActionChains(browser).send_keys(Keys.ENTER).perform()
        time.sleep(2)
        browser.quit()
    except Exception as e:
        return